In [1]:
import os

# topic_file = os.getenv("TOPIC_PATH")
# users_file = os.getenv("USERS_PATH")
# ratings_file = os.getenv("RATINGS_PATH")

#topic_file = "/data/books_data/books.csv"
#users_file = "/data/books_data/users.csv"
#ratings_file = "/data/books_data/rating.csv"

In [2]:
import pandas as pd
topics = pd.read_csv(topic_file)

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [ ]:
topics["ID"] = pd.to_numeric(topics["ID"], errors='coerce')

In [ ]:
topics.head()

In [ ]:
ratings = pd.read_csv(ratings_file)
ratings.head()

In [ ]:
users = pd.read_csv(users_file)
users.head()

In [ ]:
user_rating = pd.merge(users,ratings,on='UserID')
user_rating_mv = pd.merge(user_rating, topics, left_on="MovieID", right_on="ID") 

In [ ]:
user_rating_mv

In [ ]:
#df = pd.DataFrame().reset_index()

top_movies = user_rating_mv[['UserID','Gender','Age', 'NAME']].groupby(['Gender','Age','NAME']).count().reset_index()

In [ ]:
df1 = top_movies.query('Gender=="M" & Age=="25"').sort_values('UserID', ascending=False)
df1['NAME'].head(10)

In [ ]:
def fetch_top_movies(age, gender):
    global top_movies
    qry = 'Gender=="'+gender+'" & Age=="'+age+'"'
    df1 = top_movies.query(qry).sort_values('UserID', ascending=False)
    return df1.head(10).NAME.to_list()

In [ ]:
print(fetch_top_movies('25','F'))

In [ ]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours
import numpy
import pandas as pd

@scoring_func
def score(model, request):
    payload = request.json["payload"]
    print(payload)     
    global top_movies
    qry = 'Gender=="'+payload["gender"]+'" & Age=="'+payload["age"]+'"'
    df1 = top_movies.query(qry).sort_values('UserID', ascending=False)
    return df1.head(10).NAME.to_list()
    return prediction.tolist()

In [ ]:
modl_ob = register_model(None, score, name="Trending_Topic", 
                    description="Trending topics",
                    flavour=MLModelFlavours.sklearn,  pretty_output=False)

In [ ]:
model_id = modl_ob["id"]
version_id = modl_ob["versions"][len(modl_ob["versions"])-1]["id"]
deploy_model(model_id, version_id, "826500f6-cd56-4220-89ce-8e3370e81267")